In [2]:
#參考源代碼https://blog.csdn.net/yht2004123/article/details/82863398
import random
import pygame
import time
time1 = time.time()
# 屏幕大小的常量
SCREEN_RECT = pygame.Rect(0, 0, 480, 700)
# 刷新的頻率
FRAME_PER_SEC = 60
# 創建敵機的定時器常量
CREATE_ENEMY_EVENT = pygame.USEREVENT
# 英雄發射子彈事件
HERO_FIRE_EVENT = pygame.USEREVENT + 1


class GameSprite(pygame.sprite.Sprite):
#     """飛機大戰遊戲精靈"""

    def __init__(self,image_name,speed = 1):

        # 調用父類的初始化方法
        super().__init__()

        # 定義對象的属性
        self.image = pygame.image.load(image_name)
        self.rect = self.image.get_rect()
        self.speed = speed

    def update(self):

        # 在屏幕的垂直方向上移動
        self.rect.y += self.speed


class Background(GameSprite):
    #     """遊戲背景精靈"""

    def __init__(self, is_alt=False):

        # 1. 調用父類方法實現精灵的創建(image/rect/speed)
        super().__init__("C:/Users/USER/Desktop/夢想的價值/Python/自學/background.png")

        # 2. 判断是否是交替圖像，如果是，需要設置初始位置
        if is_alt:
            self.rect.y = -self.rect.height

    def update(self):

        # 1. 調用父類的方法實現
        super().update()

        # 2. 判断是否移出屏幕，如果移出屏幕，將圖像設置到屏幕的上方
        if self.rect.y >= SCREEN_RECT.height:
            self.rect.y = -self.rect.height


class Enemy(GameSprite):
#          """敵機精靈"""

    def __init__(self):

        # 1. 調用父類方法，創建敵機精靈，同时指定敵機圖片
        super().__init__("C:/Users/USER/Desktop/夢想的價值/Python/自學/Balloon.png")

        # 2. 指定敵機的初始隨機速度 2~5
#         self.speed = random.randint(1, 4)

        # 3. 指定敵機的初始隨機位置
        self.rect.bottom = 0

        max_x = SCREEN_RECT.width - self.rect.width
        self.rect.x = random.randint(0, max_x)

    def update(self):

        # 1. 調用父類方法，保持垂直方向的飛行
        super().update()

        # 2. 判断是否飛出屏幕，如果是，需要從精靈组删除敵機
        if self.rect.y >= SCREEN_RECT.height:
            # print("飛出屏幕，需要從精靈组删除...")
            # kill方法可以將精靈從所有精靈组中移出，精靈就會被自動銷毀
            self.kill()

    def __del__(self):
        # print("敵機掛了 %s" % self.rect)
        pass


class Hero(GameSprite):
    #     """英雄精靈"""

    def __init__(self):

        # 1. 調用父類方法，設置image&speed
        super().__init__("C:/Users/USER/Desktop/夢想的價值/Python/自學/帥.png", 0)

        # 2. 設置英雄的初始位置
        self.rect.centerx = SCREEN_RECT.centerx
        self.rect.bottom = SCREEN_RECT.bottom - 120

        # 3. 創建子彈的精靈組
        self.bullets = pygame.sprite.Group()

    def update(self):

        # 英雄在水平方向移動
        self.rect.x += self.speed

        # 控制英雄不能離開屏幕
        if self.rect.x < 0:
            self.rect.x = 0
        elif self.rect.right > SCREEN_RECT.right:
            self.rect.right = SCREEN_RECT.right

    def fire(self):
        print("發射子彈...")

        for i in (0, 1, 2):
            # 1. 創建子彈精靈
            bullet = Bullet()

            # 2. 設置精靈的位置
            bullet.rect.bottom = self.rect.y - i * 20
            bullet.rect.centerx = self.rect.centerx

            # 3. 將精靈添加到精靈組
            self.bullets.add(bullet)


class Bullet(GameSprite):
    #     """子彈精靈"""

    def __init__(self):

        # 調用父類方法，設置子弹圖片，設置初始速度
        super().__init__("C:/Users/USER/Desktop/夢想的價值/Python/自學/bullet1.png", -2)

    def update(self):

        # 調用父類方法，讓子彈沿垂直方向飛行
        super().update()

        # 判断子彈是否飛出屏幕
        if self.rect.bottom < 0:
            self.kill()

    def __del__(self):
        print("子彈被銷毀...")


    # 2.主类plane_main.py

#     import pygame
#     from plane_sprites import *


class PlaneGame(object):
    #     """飛機大戰主遊戲"""

    def __init__(self):
        print("遊戲初始化")
        # 1. 創建遊戲的窗口
        self.screen = pygame.display.set_mode(SCREEN_RECT.size)
        # 2. 創建遊戲的時鐘
        self.clock = pygame.time.Clock()
        # 3. 調用私有方法，精靈和精靈組的創建
        self.__create_sprites()

        # 4. 設置定時器事件 - 創建敵機　1s
        pygame.time.set_timer(CREATE_ENEMY_EVENT, 1000)
        pygame.time.set_timer(HERO_FIRE_EVENT, 500)

    def __create_sprites(self):

        # 創建背景精靈和精靈組
        bg1 = Background()
        bg2 = Background(True)

        self.back_group = pygame.sprite.Group(bg1, bg2)

        # 創建敵機的精靈組
        self.enemy_group = pygame.sprite.Group()

        # 創建英雄的精靈和精靈組
        self.hero = Hero()
        self.hero_group = pygame.sprite.Group(self.hero)

    def start_game(self):
        print("遊戲開始...")

        while True:
            # 1. 設置刷新頻率
            self.clock.tick(FRAME_PER_SEC)
            # 2. 事件監聽
            self.__event_handler()
            # 3. 碰撞檢測
            self.__check_collide()
            # 4. 更新/繪製精靈組
            self.__update_sprites()
            pygame.display.set_caption("MaryMin")
            # 5. 更新顯示
            pygame.display.update()

    def __event_handler(self):

        for event in pygame.event.get():

            # 判断是否退出游戲
            if event.type == pygame.QUIT:
                PlaneGame.__game_over()
            elif event.type == CREATE_ENEMY_EVENT:
                # print("敵機出場...")
                # 創建敵機精灵
                enemy = Enemy()

                # 將敵機精靈添加到敵機精靈組
                self.enemy_group.add(enemy)
            elif event.type == HERO_FIRE_EVENT:
                self.hero.fire()
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_RIGHT:
            #     print("向右移動...")

            # 使用鍵盤提供的方法獲取鍵盤按鍵 - 按鍵元组
                keys_pressed = pygame.key.get_pressed()
            # 判斷元组中對應的按鍵索引值 1
                if keys_pressed[pygame.K_RIGHT]:
                    self.hero.speed = 2
                elif keys_pressed[pygame.K_LEFT]:
                    self.hero.speed = -2
                else:
                    self.hero.speed = 0

    def __check_collide(self):

        # 1. 子彈摧毁敵机
        pygame.sprite.groupcollide(self.hero.bullets, self.enemy_group, True, True)

        # 2. 敵機撞毁英雄
        enemies = pygame.sprite.spritecollide(self.hero, self.enemy_group, True)

        # 判斷列表时候有内容
        if len(enemies) > 0:

            # 讓英雄犧牲
            self.hero.kill()

            # 結束遊戲
            PlaneGame.__game_over()

    def __update_sprites(self):

        self.back_group.update()
        self.back_group.draw(self.screen)

        self.enemy_group.update()
        self.enemy_group.draw(self.screen)

        self.hero_group.update()
        self.hero_group.draw(self.screen)

        self.hero.bullets.update()
        self.hero.bullets.draw(self.screen)

#     @staticmethod
    def __game_over():
        print("遊戲结束")

        pygame.quit()
        exit()

if __name__ == '__main__':
    
# 創建遊戲对象
    game = PlaneGame()

# 啟動遊戲
    game.start_game()
    time2 = time.time()
    print("總共消耗時間:",time2 - time1)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
遊戲初始化
遊戲開始...
發射子彈...
發射子彈...
發射子彈...
發射子彈...
發射子彈...
發射子彈...
發射子彈...
發射子彈...
發射子彈...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀...
發射子彈...
子彈被銷毀...
子彈被銷毀...
子彈被銷毀

error: display Surface quit